In [19]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
data_folder = Path("../data/raw").resolve()

amazon = DatasetReader(data_folder).get_data_csv('Amazon Sale Report.csv')
int_sales = DatasetReader(data_folder).get_data_csv('International sale Report.csv')
p_l = DatasetReader(data_folder).get_data_csv('P LMarch2021.csv')
may_2022 = DatasetReader(data_folder).get_data_csv('May-2022.csv')
sale_report = DatasetReader(data_folder).get_data_csv('Sale Report.csv')


/Users/servandodavidtorresgarcia/Servando/Relu/Relu/src/data/dataset_reader.py:13: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [34]:
expense_iigf = DatasetReader(data_folder).get_data_csv('Expense IIGF.csv')

This dataset provides an in-depth look at the profitability of e-commerce sales. It contains data on a variety of sales channels, including Shiprocket and INCREFF, as well as financial information on related expenses and profits. The columns contain data such as SKU codes, design numbers, stock levels, product categories, sizes and colors. In addition to this we have included the MRPs across multiple stores like Ajio MRP , Amazon MRP , Amazon FBA MRP , Flipkart MRP , Limeroad MRP Myntra MRP and PaytmMRP along with other key parameters like amount paid by customer for the purchase , rate per piece for every individual transaction Also we have added transactional parameters like Date of sale months category fulfilledby B2b Status Qty Currency Gross amt . This is a must-have dataset for anyone trying to uncover the profitability of e-commerce sales in today's marketplace.
This dataset provides a comprehensive overview of e-commerce sales data from different channels covering a variety of products. Using this dataset, retailers and digital marketers can measure the performance of their campaigns more accurately and efficiently.

The following steps help users make the most out of this dataset:

Analyze the general sales trends by examining info such as month, category, currency, stock level, and customer for each sale. This will give you an idea about how your e-commerce business is performing in each channel.

Review the Shiprocket and INCREF data to compare and analyze profitability via different fulfilment methods. This comparison would enable you to make better decisions towards maximizing profit while minimizing costs associated with each method’s referral fees and fulfillment rates.

Compare prices between various channels such as Amazon FBA MRP, Myntra MRP, Ajio MRP etc using  the corresponding columns for each store (Amazon MRP etc). You can judge which stores are offering more profitable margins without compromising on quality by analyzing these pricing points in combination with other information related to product sales (TP1/TP2 - cost per piece).

Look at customer specific data such as TP 1/TP 2 combination wise Gross Amount or Rate info in terms price per piece or total gross amount generated by any SKU dispersed over multiple customers with relevant dates associated to track individual item performance relative to others within its category over time periods shortlisted/filtered appropriately.. Have an eye on items commonly utilized against offers or promotional discounts offered hence crafting strategies towards inventory optimization leading up-selling operations.?

Finally Use Overall ‘Stock’ details along all the P & L Data including Yearly Expenses_IIGF information record for takeaways which might be aimed towards essential cost cutting measures like switching amongst delivery options carefully chosen out of Shiprocket & INCREFF leadings away from manual inspections catering savings under support personnel outsourcing structures.?  

By employing a comprehensive understanding on how our internal subsidiaries perform globally unless attached respective audits may provide us remarkably lower operational costs servicing confidence; costing far lesser than being incurred taking into account entire pallet shipments tracking sheets representing current level supply chains efficiencies achieved internally., then one may finally scale profits exponentially increases cut down unseen losses followed up introducing newer marketing campaigns necessarily tailored according playing around multiple goods based spectrums due powerful backing suitable transportation boundaries set carefully

In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)
amazon.head(3)
amazon.query('SKU == "SET389-KR-NP-S"').head(1
                                             )


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN


In [43]:
p_l.head()
# rename column Sku to SKU
p_l.rename(columns={'Sku': 'SKU'}, inplace=True)
may_2022.rename(columns={'Sku': 'SKU'}, inplace=True)

In [45]:
print(amazon.SKU.nunique())
print(p_l.SKU.nunique())
print(may_2022.SKU.nunique())

7195
1330
1330


In [50]:
may_2022.tail()
#may_2022.query('Sku == "SET389-KR-NP-S"')
amazon.query('SKU == "SET389-KR-NP-S"').head(1)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN


In [51]:
# Create a new feature called sales that is the  the count of "Amount" grouped by "SKU" in the amazon dataset
amazon['sales'] = amazon.groupby('SKU')['Amount'].transform('count')

In [58]:
amazon.query('SKU == "SET389-KR-NP-S"')
#amazon.info()
# drop columns: index, Order ID, ASIN, Unnamed: 22
amazon.drop(columns=[ 'Order ID', 'ASIN', 'Unnamed: 22'], inplace=True)

In [60]:
amazon.drop(columns=[ 'index'], inplace=True)

In [62]:
amazon.head(3)
# change Date column to datetime
amazon['Date'] = pd.to_datetime(amazon['Date'])

In [67]:
amazon.head()
amazon['ship-postal-code'] = amazon['ship-postal-code'].astype('object')

In [68]:
amazon.head(2)

,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,sales
0,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,35
1,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,101


In [70]:
from src.data.eda import Eda
eda = Eda(    
)

eda.missing_values_table(amazon)

The selected dataframe has 21 columns and 9 columns with missing values.


,Missing Values,% of Total Values
fulfilled-by,89698,69.5
promotion-ids,49153,38.1
currency,7795,6.0
Amount,7795,6.0
Courier Status,6872,5.3
ship-city,33,0.0
ship-state,33,0.0
ship-postal-code,33,0.0
ship-country,33,0.0


In [71]:
amazon.head(2)

,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,sales
0,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,NaN,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,35
1,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,101


In [122]:
amazon_beta = amazon[['Date', 'SKU', 'Qty', 'Amount', 'sales']]

In [123]:
amazon_beta.rename(columns={'Amount': 'price'}, inplace=True)

In [82]:
amazon_beta.head(2)
amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by='Date', ascending=True).head()

,Date,SKU,Qty,price,sales
47784,2022-04-01,SET389-KR-NP-S,1,749.00,35
41605,2022-04-05,SET389-KR-NP-S,1,0.00,35
36996,2022-04-08,SET389-KR-NP-S,1,680.00,35
18468,2022-04-19,SET389-KR-NP-S,1,680.00,35
16932,2022-04-20,SET389-KR-NP-S,0,647.62,35


In [124]:
# show duplicates in amazon_beta 
#amazon_beta[amazon_beta.duplicated(keep=False)].sort_values(by='SKU', ascending=True).head(10)
# drop duplicates in amazon_beta
amazon_beta.drop_duplicates(inplace=True)

In [125]:
amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by='Date', ascending=True).head()

,Date,SKU,Qty,price,sales
47784,2022-04-01,SET389-KR-NP-S,1,749.00,35
41605,2022-04-05,SET389-KR-NP-S,1,0.00,35
36996,2022-04-08,SET389-KR-NP-S,1,680.00,35
18468,2022-04-19,SET389-KR-NP-S,1,680.00,35
16932,2022-04-20,SET389-KR-NP-S,0,647.62,35


In [126]:
eda.missing_values_table(amazon_beta)

The selected dataframe has 5 columns and 1 columns with missing values.


,Missing Values,% of Total Values
price,6957,7.6


In [127]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [132]:
import pandas as pd

def calculate_rolling_average_price(dataset, window=13):
    """
    Calculate the average price for the last `window` days grouped by SKU.
    
    Parameters:
    - dataset: DataFrame with the columns ['Date', 'SKU', 'Qty', 'price', 'sales'].
    - window: Integer, the number of days to calculate the rolling average.
    
    Returns:
    - DataFrame with a new column 'avg_price_30' containing the rolling average prices.
    """
    
    # Sort by SKU and Date
    dataset = dataset.sort_values(by=['SKU', 'Date'])

    # Set the date as the index for rolling operation
    dataset = dataset.set_index('Date')
    
    # Calculate the rolling average price grouped by SKU
    def rolling_avg(x):
        return x.rolling(window, min_periods=1).mean()
        
    dataset['avg_price_N'] = dataset.groupby('SKU')['price'].apply(rolling_avg)

    # Reset the index
    dataset = dataset.reset_index()
    
    return dataset






In [133]:
amazon_beta = calculate_rolling_average_price(amazon_beta, window=13)

In [135]:
amazon_beta.head()

,Date,SKU,Qty,price,sales,avg_price_N
0,2022-04-07,AN201-RED-M,1,229.0,2,229.0
1,2022-05-01,AN201-RED-M,1,229.0,2,229.0
2,2022-06-24,AN201-RED-XL,1,301.0,2,301.0
3,2022-06-26,AN201-RED-XL,1,301.0,2,301.0
4,2022-05-03,AN201-RED-XXL,1,229.0,1,229.0


In [136]:
def filter_rows(group):
    return group[group['price'] != group['avg_price_N']]

amazon_beta_filtered = amazon_beta.groupby('SKU').apply(filter_rows).reset_index(drop=True)

In [ ]:
# calculate the average number of days